In [1]:
# dados do problema

import pandas as pd

periodos = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Periods')
plantas = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Plants', decimal=",")
linhas = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Lines')
cds = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Dist. Centers', decimal=",")
clientes = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Customers', decimal=",")
produtos = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Products')
rotas = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Routes', decimal=",")
demandas = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Demands')
capabilidades = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Capabilities')
ktaxas = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Rates')

#Conjuntos

In [2]:
P = plantas['name'].tolist() # set plants
W = cds['name'].tolist() # set warehouses
Lines = linhas['name'].tolist() # set Line names
plants_of_lines = linhas['plant'].tolist()

In [3]:
p_l = dict(zip(Lines,plants_of_lines))

k = {}
for _, i in ktaxas.iterrows():
  # var = i.to_dict()
  line, period, size, rate = i['line'], i['period'], i['size'], i['rate']
  k[(size,line,period)] = rate

C = clientes['name'].tolist()
I = produtos['product'].tolist()
E = produtos['size'].tolist()
T = periodos['periods'].tolist()
size_of_products = dict(zip(I,E)) #tamanho de cada produto
tamanhos = list(set(E))
products_per_size = {i:[] for i in tamanhos} # produtos por tamanho
for i in I:
  products_per_size[size_of_products[i]].append(i)


In [4]:
print(p_l)

{'Plant 0_1': 'Plant 0', 'Plant 0_2': 'Plant 0', 'Plant 1_1': 'Plant 1', 'Plant 1_2': 'Plant 1', 'Plant 1_3': 'Plant 1', 'Plant 1_4': 'Plant 1', 'Plant 2_1': 'Plant 2', 'Plant 2_2': 'Plant 2', 'Plant 2_3': 'Plant 2'}


In [5]:
#relate products to sizes

def prod_size(prod=None, size=None):
  if prod != None:
    return size_of_products[prod]
  elif size != None:
    return products_per_size[size]

#Parametros

In [6]:
G_p = dict(zip(P, list(plantas['storage_capacity']))) # capacidade max das plantas
G_w = dict(zip(W, list(cds['storage_capacity']))) # capacidade max dos warehouses
taxas = k # taxas por linha linha plant0_1:{periodo_t:{size:12oz, rate:k}}
print(taxas)

{('24oz', 'Plant 0_1', Timestamp('2025-04-14 00:00:00')): 2892235, ('24oz', 'Plant 0_1', Timestamp('2025-04-15 00:00:00')): 2879392, ('24oz', 'Plant 0_1', Timestamp('2025-04-16 00:00:00')): 3010741, ('24oz', 'Plant 0_1', Timestamp('2025-04-17 00:00:00')): 3000727, ('24oz', 'Plant 0_1', Timestamp('2025-04-18 00:00:00')): 2732801, ('24oz', 'Plant 0_1', Timestamp('2025-04-19 00:00:00')): 2840744, ('24oz', 'Plant 0_1', Timestamp('2025-04-20 00:00:00')): 2976485, ('24oz', 'Plant 0_2', Timestamp('2025-04-14 00:00:00')): 3060070, ('24oz', 'Plant 0_2', Timestamp('2025-04-15 00:00:00')): 3068860, ('24oz', 'Plant 0_2', Timestamp('2025-04-16 00:00:00')): 3137949, ('24oz', 'Plant 0_2', Timestamp('2025-04-17 00:00:00')): 2745648, ('24oz', 'Plant 0_2', Timestamp('2025-04-18 00:00:00')): 3050635, ('24oz', 'Plant 0_2', Timestamp('2025-04-19 00:00:00')): 3131678, ('24oz', 'Plant 0_2', Timestamp('2025-04-20 00:00:00')): 2937343, ('24oz', 'Plant 1_1', Timestamp('2025-04-14 00:00:00')): 2998641, ('24oz', 

In [7]:
t_pc = {} # taxa de frete | indexado por t_pc[(planta_p, cliente_c)]
for _,i in rotas.iterrows():
  if i.origin[0] == "P" and i.destination[0] == "C":
    org, dest, cost = i.origin, i .destination, i.unit_freight_cost
    t_pc[(org, dest)] = cost

print(t_pc)

{('Plant 0', 'Customer 0'): 0.03132078, ('Plant 0', 'Customer 1'): 0.038193328, ('Plant 0', 'Customer 2'): 0.033438732000000006, ('Plant 0', 'Customer 3'): 0.027730024000000002, ('Plant 0', 'Customer 4'): 0.063178612, ('Plant 0', 'Customer 5'): 0.017336628, ('Plant 0', 'Customer 6'): 0.061580816, ('Plant 0', 'Customer 7'): 0.020572824, ('Plant 0', 'Customer 9'): 0.028110952, ('Plant 0', 'Customer 10'): 0.031995192, ('Plant 0', 'Customer 11'): 0.049305288, ('Plant 0', 'Customer 12'): 0.047474864, ('Plant 1', 'Customer 0'): 0.011847752, ('Plant 1', 'Customer 1'): 0.041234248, ('Plant 1', 'Customer 2'): 0.02703234, ('Plant 1', 'Customer 3'): 0.05345196, ('Plant 1', 'Customer 4'): 0.086970092, ('Plant 1', 'Customer 5'): 0.036856836000000004, ('Plant 1', 'Customer 6'): 0.09281761599999999, ('Plant 1', 'Customer 7'): 0.029113164, ('Plant 1', 'Customer 9'): 0.065816504, ('Plant 1', 'Customer 10'): 0.053561824, ('Plant 1', 'Customer 11'): 0.062790612, ('Plant 1', 'Customer 13'): 0.019236776, (

In [8]:
t_wc = {} # taxa de frete | indexado por t_wc[(warehouse_w, cliente_c)]
for _,i in rotas.iterrows():
  if i.origin[0] == "W" and i.destination[0] == "C":
    org, dest, cost = i.origin, i .destination, i.unit_freight_cost
    t_wc[(org, dest)] = cost
print(t_wc)

{('Warehouse 1', 'Customer 0'): 0.026636144, ('Warehouse 2', 'Customer 0'): 0.030915704000000002, ('Warehouse 3', 'Customer 0'): 0.02297954, ('Warehouse 1', 'Customer 1'): 0.031869916, ('Warehouse 2', 'Customer 1'): 0.042318072, ('Warehouse 3', 'Customer 1'): 0.038155131999999994, ('Warehouse 1', 'Customer 2'): 0.031893072, ('Warehouse 2', 'Customer 2'): 0.033113932, ('Warehouse 3', 'Customer 2'): 0.033737312, ('Warehouse 1', 'Customer 3'): 0.029804492, ('Warehouse 2', 'Customer 3'): 0.03816628, ('Warehouse 3', 'Customer 3'): 0.041329884, ('Warehouse 1', 'Customer 4'): 0.069335256, ('Warehouse 2', 'Customer 4'): 0.061773472, ('Warehouse 3', 'Customer 4'): 0.063669688, ('Warehouse 1', 'Customer 5'): 0.016974867999999997, ('Warehouse 2', 'Customer 5'): 0.02251602, ('Warehouse 3', 'Customer 5'): 0.020371692, ('Warehouse 1', 'Customer 6'): 0.061628235999999996, ('Warehouse 2', 'Customer 6'): 0.0600151, ('Warehouse 3', 'Customer 6'): 0.058026716, ('Warehouse 1', 'Customer 7'): 0.016033748, 

In [9]:
t_pp = {} # taxa de frete | indexado por t_pp[(planta_p, planta_q)]
for _,i in rotas.iterrows():
  if i.origin[0] == "P" and i.destination[0] == "P":
    org, dest, cost = i.origin, i .destination, i.unit_freight_cost
    t_pp[(org, dest)] = cost

print(t_pp)

{('Plant 0', 'Plant 1'): 0.038653184, ('Plant 0', 'Plant 2'): 0.01329084, ('Plant 1', 'Plant 0'): 0.039087620000000003, ('Plant 1', 'Plant 2'): 0.026028604, ('Plant 2', 'Plant 0'): 0.012114724, ('Plant 2', 'Plant 1'): 0.028044707999999998}


In [10]:
t_pw = {} # taxa de frete | indexado por t_pp[(planta_p, planta_q)]
for _,i in rotas.iterrows():
  if i.origin[0] == "P" and i.destination[0] == "W":
    org, dest, cost = i.origin, i .destination, i.unit_freight_cost
    t_pw[(org, dest)] = cost

print(t_pw)

{('Plant 0', 'Warehouse 1'): 0.004029489760321267, ('Plant 0', 'Warehouse 2'): 0.004509939923368799, ('Plant 0', 'Warehouse 3'): 0.0072410703145020915, ('Plant 1', 'Warehouse 4'): 0.005161979430361879, ('Plant 1', 'Warehouse 5'): 0.005139100851169139}


In [11]:
D = {} # Demandas | indexado por D[(customer_c, product_i, period_t)]
for _,r in demandas.iterrows():
  client, prod, perio , demand = r['customer'], r['product'], r['period'], r['demand']
  D[(client, prod, perio)] = demand
print(D)


{('Customer 0', 'Product 7', Timestamp('2025-04-14 00:00:00')): 1600000, ('Customer 0', 'Product 19', Timestamp('2025-04-15 00:00:00')): 1400000, ('Customer 0', 'Product 16', Timestamp('2025-04-16 00:00:00')): 400000, ('Customer 0', 'Product 21', Timestamp('2025-04-16 00:00:00')): 600000, ('Customer 0', 'Product 19', Timestamp('2025-04-17 00:00:00')): 800000, ('Customer 0', 'Product 21', Timestamp('2025-04-18 00:00:00')): 1600000, ('Customer 0', 'Product 21', Timestamp('2025-04-19 00:00:00')): 1000000, ('Customer 0', 'Product 25', Timestamp('2025-04-19 00:00:00')): 800000, ('Customer 0', 'Product 22', Timestamp('2025-04-20 00:00:00')): 1000000, ('Customer 1', 'Product 8', Timestamp('2025-04-14 00:00:00')): 800000, ('Customer 1', 'Product 5', Timestamp('2025-04-14 00:00:00')): 400000, ('Customer 1', 'Product 14', Timestamp('2025-04-15 00:00:00')): 1000000, ('Customer 1', 'Product 12', Timestamp('2025-04-16 00:00:00')): 1000000, ('Customer 1', 'Product 19', Timestamp('2025-04-17 00:00:00

In [12]:
holding_cost = 0.1
M = 10 # penalizacao por falta a demanda
truck_capacity = 250000
estoque_inicial_p = {i:0 for i in P}
estoque_inicial_w = {i:0 for i in W}

calcular custo de frete de planta para armazem através de distância

In [13]:
from pyomo.environ import *
model = ConcreteModel()

#inicializa os conjuntos
model.P = Set(initialize=P) # plantas
model.L = Set(initialize=Lines) # Linhas
model.W = Set(initialize=W) # armazens
model.C = Set(initialize=C) # clientes
model.I = Set(initialize=I) # produtos
model.T = Set(initialize=T) # periodos
model.E = Set(initialize=tamanhos) # tamanhos

#inicializa os parâmetros
model.G_w = Param(model.W, initialize=G_w) # capacidade maxima de estoque no armzem w
model.G_p = Param(model.P, initialize=G_p) # capacidade maxima de estoque na planta p
model.h = Param(initialize=holding_cost) # custo de estoque por unidade por periodo
model.M = Param(initialize=M) # penalizador de demanda
model.Q = Param(initialize=truck_capacity) # capacidade maxima por caminhao
model.k = Param(model.E,model.L,model.T, initialize=taxas, default= 0) # capacidade produtiva de cada linha
model.t_pp = Param(model.P, model.P, initialize=t_pp, default=1e20) # custos  de frete
model.t_pc = Param(model.P, model.C, initialize=t_pc, default=1e20) # custos de frete
model.t_wc = Param(model.W, model.C, initialize=t_wc, default=1e20) # custos de frete
model.t_pw = Param(model.P, model.W, initialize=t_pw, default=1e20) #custos de frete
model.D = Param(model.C, model.I,model.T, initialize=D, default=0) # demandas
model.estoque_inicial_p = Param(model.P, initialize=estoque_inicial_p)
model.estoque_inicial_w = Param(model.W, initialize=estoque_inicial_w)

#incializar as variaveis

model.x = Var(model.I , model.L , model.T, domain=NonNegativeReals)
model.x_p = Var(model.I , model.P , model.T , domain=NonNegativeReals)
model.S_p = Var(model.I, model.P, model.T, domain=NonNegativeReals)
model.S_w = Var(model.I, model.W, model.T, domain=NonNegativeReals)
model.f_pc = Var(model.I, model.P, model.C, model.T, domain=NonNegativeReals)
model.f_wc = Var(model.I, model.W, model.C, model.T, domain=NonNegativeReals)
model.f_pp = Var(model.I, model.P, model.P, model.T ,domain=NonNegativeReals)
model.f_pw = Var(model.I, model.P, model.W, model.T  ,domain=NonNegativeReals)
model.U = Var(model.C, model.I, model.T, domain=NonNegativeReals)

# NOVO: Variaveis inteiras para o número de caminhões
# model.y_pc = Var(model.I, model.P, model.C, model.T, domain=NonNegativeIntegers)
# model.y_wc = Var(model.I, model.W, model.C, model.T, domain=NonNegativeIntegers)
# model.y_pp = Var(model.I, model.P, model.P, model.T ,domain=NonNegativeIntegers)
# model.y_pw = Var(model.I, model.P, model.W, model.T  ,domain=NonNegativeIntegers)

In [14]:
# definir funcao objetivo

def total_cost(m):
      # custo de frete Planta-Cliente
      cost_f_pc = sum(m.t_pc[p,c]* m.f_pc[i,p,c,t]
                      for i in m.I for p in m.P for c in m.C for t in m.T)
      # custo de frete Armazem-Cliente
      cost_f_wc = sum(m.t_wc[w, c] * m.f_wc[i, w, c, t]
                    for i in m.I for w in m.W for c in m.C for t in m.T)

      # custo de frete Planta-Planta
      cost_f_pp = sum(m.t_pp[p1, p2] * m.f_pp[i, p1, p2, t]
                    for i in m.I for p1 in m.P for p2 in m.P if p1 != p2 for t in m.T) # Adicionado if p1 != p2 para evitar frete de planta para ela mesma

      # custo de frete Planta-Armazem
      cost_f_pw = sum(m.t_pw[p, w] * m.f_pw[i, p, w, t]
                    for i in m.I for p in m.P for w in m.W for t in m.T)

      # custo de estoque no Armazem
      cost_h_w = sum(m.h * m.S_w[i,w,t]
                     for i in m.I for w in m.W for t in m.T)

      # custo de penalidade por demanda nao atendida
      cost_unmet_demand = m.M * sum(m.U[c,i,t]
                                    for c in m.C for i in m.I for t in m.T)

      return cost_f_pc + cost_f_wc + cost_f_pp + cost_f_pw + cost_h_w + cost_unmet_demand


model.obj = Objective(rule=total_cost, sense=minimize)


In [15]:
# definir as restrições

# 1) restricao de producao
model.Producao = ConstraintList()
for t in model.T:
  for l in model.L:
      for e in model.E:
        model.Producao.add(expr = sum(model.x[i,l,t] for i in model.I if size_of_products.get(i) == e) <= model.k[e,l,t])

# Restrição para definir x_p como a soma da produção por planta
model.Definicao_x_p = ConstraintList()
for i in model.I:
  for p in model.P:
    for t in model.T:
      model.Definicao_x_p.add(expr = model.x_p[i, p, t] == sum(model.x[i, l, t] for l in model.L if p_l[l] == p))



# 2) restricao de balanco de estoque na planta
# Para o primeiro período (t=1), consideramos o estoque inicial como 0
model.Balanco_estoque_planta = ConstraintList()
for p in model.P:
  for i in model.I:
    # Assuming T starts from 1
    first_period = min(model.T)
    model.Balanco_estoque_planta.add(
        expr = model.estoque_inicial_p[p] + model.x_p[i, p, first_period] + sum(model.f_pp[i, q, p, first_period] for q in model.P if q != p) ==
               model.S_p[i, p, first_period] + sum(model.f_pw[i, p, w, first_period] for w in model.W) + sum(model.f_pc[i, p, c, first_period] for c in model.C) + sum(model.f_pp[i, p, q, first_period] for q in model.P if q != p)
    )

# Para os períodos subsequentes (t > 1)
for t in model.T:
  if t > min(model.T):
    previous_period = list(model.T)[list(model.T).index(t) - 1]
    for p in model.P:
      for i in model.I:
        model.Balanco_estoque_planta.add(
            expr = model.S_p[i, p, previous_period] + sum(model.x_p[i, p, t] for l in model.L) + sum(model.f_pp[i, q, p, t] for q in model.P if q != p) ==
                   model.S_p[i, p, t] + sum(model.f_pw[i, p, w, t] for w in model.W) + sum(model.f_pc[i, p, c, t] for c in model.C) + sum(model.f_pp[i, p, q, t] for q in model.P if q != p)
        )


# 3)restricao de balanco de estoque no armazem
# Para o primeiro período (t=1), consideramos o estoque inicial como 0
model.Balanco_estoque_armazem = ConstraintList()
first_period = min(model.T)
for w in model.W:
  for i in model.I:
    model.Balanco_estoque_armazem.add(
        expr =  model.estoque_inicial_w[w] + sum(model.f_pw[i, p, w, first_period] for p in model.P) ==
               model.S_w[i, w, first_period] + sum(model.f_wc[i, w, c, first_period] for c in model.C)
    )

# Para os períodos subsequentes (t > 1)
for t in model.T:
  if t > min(model.T):
    previous_period = list(model.T)[list(model.T).index(t) - 1]
    for w in model.W:
      for i in model.I:
        model.Balanco_estoque_armazem.add(
            expr = model.S_w[i, w, previous_period] + sum(model.f_pw[i, p, w, t] for p in model.P) ==
                   model.S_w[i, w, t] + sum(model.f_wc[i, w, c, t] for c in model.C)
        )


# 4) restricao de estoque maximo na planta
model.Estoque_maximo_p = ConstraintList()
for p in model.P:
  for t in model.T:
    model.Estoque_maximo_p.add(
        expr = sum(model.S_p[i, p, t] for i in model.I) <= model.G_p[p]
    )


# 5) restricao de estoque maximo no armazem
model.Estoque_maximo_w = ConstraintList()
for w in model.W:
  for t in model.T:
    model.Estoque_maximo_w.add(
        expr = sum(model.S_w[i, w, t] for i in model.I) <= model.G_w[w]
    )


# 6) restricao de demanda
model.Demanda = ConstraintList()
for c in model.C:
  for i in model.I:
    for t in model.T:
      model.Demanda.add(
          expr = sum(model.f_pc[i, p, c, t] for p in model.P) + sum(model.f_wc[i, w, c, t] for w in model.W) + model.U[c,i,t] >= model.D[c, i, t]
          )

In [16]:
import highspy

In [17]:
solver = SolverFactory('highs')

solver.solve(model)
# Obtém o valor do objetivo
objective_value = model.obj()
print(f"Valor do objetivo : {objective_value:.2f} ")

unmet_demand_total = 0
unmet_demands = {}

u_df = {"Client" : [],"Product" : [], "Period": [], "Unmet Demand" : []}

# print("Valores das variáveisc U[c,i,t]:")
for c in model.C:
  for i in model.I:
    for t in model.T:
      U = model.U[c,i,t].value
      unmet_demand_total += U
      if U != 0:
        u_df["Client"].append(c), u_df["Product"].append(i), u_df["Period"].append(t) , u_df["Unmet Demand"].append(U)
        unmet_demands[(c,i,t)] = U
        print(f"U[{c},{i},{t}] = {U}")




Valor do objetivo : 160337471.78 
U[Customer 0,Product 7,2025-04-14 00:00:00] = 1600000.0
U[Customer 1,Product 8,2025-04-14 00:00:00] = 800000.0
U[Customer 2,Product 7,2025-04-14 00:00:00] = 1000000.0
U[Customer 2,Product 16,2025-04-14 00:00:00] = 1400000.0
U[Customer 2,Product 19,2025-04-14 00:00:00] = 1200000.0
U[Customer 5,Product 10,2025-04-14 00:00:00] = 1400000.0
U[Customer 8,Product 10,2025-04-14 00:00:00] = 1000000.0
U[Customer 9,Product 7,2025-04-14 00:00:00] = 800000.0
U[Customer 10,Product 19,2025-04-14 00:00:00] = 1000000.0
U[Customer 12,Product 18,2025-04-14 00:00:00] = 1000000.0
U[Customer 12,Product 20,2025-04-14 00:00:00] = 1400000.0
U[Customer 14,Product 7,2025-04-14 00:00:00] = 1400000.0
U[Customer 14,Product 17,2025-04-14 00:00:00] = 1200000.0


In [18]:
print(f"Total unmet demand: {unmet_demand_total}")
print(f"Total cost of operation: {objective_value - unmet_demand_total*M}")

Total unmet demand: 15200000.0
Total cost of operation: 8337471.781093687


In [19]:
print("Unmet Demands: ")
unmet_demands
u_df = pd.DataFrame(u_df)
u_df

Unmet Demands: 


,Client,Product,Period,Unmet Demand
0,Customer 0,Product 7,2025-04-14,1600000.0
1,Customer 1,Product 8,2025-04-14,800000.0
2,Customer 2,Product 7,2025-04-14,1000000.0
3,Customer 2,Product 16,2025-04-14,1400000.0
4,Customer 2,Product 19,2025-04-14,1200000.0
5,Customer 5,Product 10,2025-04-14,1400000.0
6,Customer 8,Product 10,2025-04-14,1000000.0
7,Customer 9,Product 7,2025-04-14,800000.0
8,Customer 10,Product 19,2025-04-14,1000000.0
9,Customer 12,Product 18,2025-04-14,1000000.0


In [20]:
u_df.to_excel("./sample_data/unmet_demand.xlsx", index=False)

In [23]:
x_df = {'plant':[] , 'period':[], 'product':[], 'production':[]}

for p in model.P:
  for t in model.T:
    for i in model.I:
      if model.x_p[i,p,t].value != 0:
        x_df['production'].append(model.x_p[i,p,t].value), x_df['plant'].append(p), x_df['product'].append(i), x_df['period'].append(t)

x_df = pd.DataFrame(x_df)
x_df

,plant,period,product,production
0,Plant 0,2025-04-14,Product 21,4.000000e+05
1,Plant 0,2025-04-14,Product 22,4.000000e+05
2,Plant 0,2025-04-14,Product 23,1.400000e+06
3,Plant 0,2025-04-14,Product 24,2.000000e+06
4,Plant 0,2025-04-14,Product 25,1.200000e+06
...,...,...,...,...
110,Plant 2,2025-04-20,Product 6,4.444444e+04
111,Plant 2,2025-04-20,Product 7,2.444444e+05
112,Plant 2,2025-04-20,Product 10,1.555556e+05
113,Plant 2,2025-04-20,Product 23,1.333333e+05


In [22]:
x_df.to_excel("./sample_data/production.xlsx" ,index=False)